In [1]:
import sys
from pathlib import Path

import pandas as pd
import os 
import numpy as np

TOP_DIR = Path('../../../').resolve()

if str(TOP_DIR) not in sys.path: sys.path.append(str(TOP_DIR))

SITE_DATA_DIR = '../../../docs/metrics/partnerships/_data'

PARTNERSHIPS_DATA = '../../../data/metrics/partnerships/partnerships.csv'
COUNTRIES_DATA = '../../../data/metrics/partnerships/countries.csv'

In [2]:
os.makedirs(SITE_DATA_DIR, exist_ok=True)

partnerships_data = pd.read_csv(PARTNERSHIPS_DATA, parse_dates=['start_date'])
countries_data = pd.read_csv(COUNTRIES_DATA)

Summarise

In [5]:
pd.Series({
    'total_partnerships': len(partnerships_data),
    'total_international': partnerships_data.international_element_to_project_.count(),
    'total_unique_countries': len(countries_data.unique_countries),
    'total_events': partnerships_data.total_number_of_events.sum(),
    'total_audience': partnerships_data.audience_numbers.sum(),
    'total_participants': partnerships_data.number_of_participants.sum(),
    'total_in_person_events': partnerships_data['online_in_person_both'].value_counts()['In person'],
    'both_online_in_person_events': partnerships_data['online_in_person_both'].value_counts()['Both'],
    'online_only_events': partnerships_data['online_in_person_both'].value_counts()['Online'],
    'project_performance': partnerships_data['project_output'].value_counts()['Exhibition']
}).astype(int).to_json(
    os.path.join(SITE_DATA_DIR, 'headlines.json'),
    date_format='iso', indent=2
)

AttributeError: 'str' object has no attribute 'value_counts'

Summarise by ward

In [ ]:
partnerships_data.fillna('UNKNOWN').groupby('partner_venue_ward_code').partner_organisation.count().to_csv(os.path.join(SITE_DATA_DIR, 'partnerships_by_ward.csv'))

Summarise by month

In [ ]:
partnerships_data.groupby('start_date').partner_organisation.count().resample('MS').sum().cumsum().to_csv(os.path.join(SITE_DATA_DIR, 'partnerships_by_month.csv'))

Project outputs

In [8]:
partnerships_data['project_output'] = (partnerships_data.project_output.explode().loc[partnerships_data['project_output'].isin(['Performance'])]).count()
partnerships_data




,partner_organisation,list_of_countries_involved,end_date,start_date,partner_venue_postcode,partnership_status,project_artform,project_name,project_output,international_element_to_project_,online_in_person_both,number_of_countries_involved,season,duration_of_activity_event,contains_heritage_young_people,digital_audiences,audience_numbers,number_of_participants,total_number_of_events,partner_venue_ward_code
0,Leeds Art Gallery,['Italy'],2023-10-05,2023-10-05,LS1 6NU,In progress,Music,Leeds Art Gallery: Sonia Boyce (HAR Event),NaN,True,In person,1.0,Dreaming (1 Sep - 31 Dec),['One off event/activity'],NaN,NaN,NaN,NaN,NaN,E05011403
1,Somone (Band Name),['Netherlands'],2023-10-17,2023-10-17,LS28 5UJ,Done,Music,Someone at The Old Woollen,NaN,True,In person,1.0,Dreaming (1 Sep - 31 Dec),['One off event/activity'],NaN,NaN,NaN,NaN,NaN,E05011389
2,The Performance Ensemble,"['China', 'Hong Kong']",2023-05-06,2023-04-24,LS2 7UP,In progress,Combined arts,The Performance Ensemble: 1001 Stories & Sinfo...,NaN,True,In person,2.0,Awakening (1 Jan - 30 Apr),['Series of events/activities'],NaN,NaN,NaN,NaN,NaN,E05011403
3,Leeds Art Gallery & Yorkshire Dance,['Netherlands'],2023-12-10,2023-12-08,LS1 3AA,In progress,Dance,MEET LEEDS LAB: Katjia Heitmann: Motus Mori Co...,NaN,True,In person,1.0,Dreaming (1 Sep - 31 Dec),['Series of events/activities'],NaN,NaN,NaN,NaN,NaN,E05011403
4,Daily Lift Ltd,NaN,2023-12-17,2023-12-13,LS10 1JQ,In progress,Combined arts,Daily Life Ltd: EPIC DOMESTIC,NaN,NaN,In person,0.0,Dreaming (1 Sep - 31 Dec),['Series of events/activities'],NaN,NaN,NaN,NaN,NaN,E05011399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,East Street Arts,['Netherlands'],2023-01-20,2023-01-16,LS9 7EH,Done,Combined arts,MEET LEEDS LAB: January 2023,NaN,True,In person,1.0,Awakening (1 Jan - 30 Apr),['Series of events/activities'],NaN,NaN,62.0,9.0,NaN,E05011388
62,Sunken Studio,['Netherlands'],2023-10-20,2023-12-01,LS101AW,In progress,Combined arts,MEET LEEDS LAB: Archie Hyde (Artistic Residency),NaN,True,In person,1.0,Dreaming (1 Sep - 31 Dec),['Series of events/activities'],NaN,NaN,NaN,NaN,1.0,E05011399
63,Rugby League World Cup,NaN,2023-09-28,2022-10-13,M3 4AP,Done,Combined arts,Rugby League World Cup 2021,NaN,NaN,In person,0.0,2022,['Series of events/activities'],['Young people'],NaN,NaN,NaN,NaN,E05011361
64,Opera North,['South Africa'],2023-04-29,2023-03-30,LS2 7AU,In progress,Music,Opera North: As You Are,NaN,True,In person,1.0,Awakening (1 Jan - 30 Apr),['Series of events/activities'],NaN,NaN,1966.0,NaN,NaN,E05011403
